#  BI-Directional Long Short Term Memory Model (BI-LSTM)

**What is Bi-Directional Lstm ?**
- Bi-Directional LSTM is a type of Sequence Model that Consists of two LSTM Layers.
- One Processes the  input in the forward Direction and the other processes it in the Backward Direction.
The architecture of a Bi Directional LSTM  includes two Separate LSTM networks:

**Forward LSTM:** 
- Process the Sequence from start to end.

**Backward LSTM:** 
- Processes the Sequence from end to start.


The Below are the Required Modules for this Model

In [1]:
# Required Models for the project

import pandas as pd # type: ignore
import numpy as  np # type: ignore
import os # type: ignore
import math # type: ignore
import matplotlib.pyplot as plt # type: ignore

from sklearn.preprocessing import MinMaxScaler # type: ignore
from tensorflow.keras.models import Sequential, save_model, load_model  # type: ignore
from tensorflow.keras.layers import InputLayer, Bidirectional, LSTM, Dense # type: ignore
from tensorflow.keras.losses import MeanSquaredError # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error # type: ignore

Now, we have to load our dataset required for the Model

In [ ]:
file_path = 'PATH_TO_YOUR_DATASET.csv' # Path to your dataset
dataset = pd.read_csv(file_path) # Load the dataset

To make sure There is no zero values in the dataset,we have to check is there any null values in the dataset.

In [ ]:
null_values = dataset.isnull().sum() # Check for null values in the dataset
print("Null values in each column:\n", null_values) # Print the null values in the dataset

Now, We have to get the required data values in between 0 and 1 to make the caculations simpler.

So, we are using the Normalizatioin Method.

In [ ]:
column = ['Column Name'] # Column name to be normalized
scaler = MinMaxScaler() # Normalizing the data
dataset[column] = scaler.fit_transform(dataset[column]) # Normalizing the data
dataset.head() # Displaying the first 5 rows of the dataset

# Sequence Creation

We have to divide the data into the sequences according to the required number of TimeIntervals.

In [ ]:
# Function to create sequences

def create_sequences(column_data, time_steps):
    X, Y = [], []
    for i in range(len(column_data) - time_steps):
        X.append(column_data[i:i + time_steps])
        Y.append(column_data[i + time_steps])
    return np.array(X), np.array(Y)

In [ ]:
column_data = dataset[['Column_Name']].values
time_steps = 'No_of_time_steps' # Number of time steps   
input_dimension = 1 # Input dimension

BI_LSTM_X, y = create_sequences(column_data, time_steps) # Creating sequences

# Displaying the values of LSTM_X and y

print(f"Values of BI_LSTM_X: {BI_LSTM_X}") 
print(f"Values of y: {y}")

print(f"Shape of BI_LSTM_X: {BI_LSTM_X.shape}")
print(f"Shape of y: {y.shape}")

# Spliting the Data 

Now , we have to split the data for training and testing the data.

In [ ]:
# Splitting the data into training top 90% and testing bottom 10%

train_size = int(len(BI_LSTM_X) * 0.90) 
X_train, X_test = BI_LSTM_X[:train_size], BI_LSTM_X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Displaying the shapes of the training and testing data

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

print("Training X Values : ",X_train)
print("Testing X Values : ",X_test)
print("Training y Values : ",y_train)
print("Testing y Values : ",y_test)

# BI-LSTM Model Creation

We have to create the BI-LSTM Model as we already imported the required modeules respectively.

In [ ]:
model = Sequential() # Creating a Sequential model
model.add(Bidirectional(LSTM('Number of Neurons Required', return_sequences=True), input_shape=(time_steps, 1))) # Adding a Bidirectional LSTM layer
model.add(Bidirectional(LSTM('Number of Neurons Required', return_sequences=False))) # Adding a Bidirectional LSTM layer
model.add(Dense('Number of Neurons Required', activation='relu')) # Adding a Dense layer
model.add(Dense(1)) # Adding a Dense layer
model.compile(optimizer='', loss='') # Compiling the model
model.summary() # Displaying the model summary

We created the model for the LSTM, Now we have to train the model by using the Training Data.

In [ ]:
history=model.fit(X_train, y_train,epochs='Number of Iterations', batch_size=32, validation_split="mention the validation split value", verbose=1) # Fitting the model

Upto to this is the model creation and the model training.

Let's Predict the values using the model we trained.

In [ ]:
train_predictions = model.predict("x_testing_data_values") # Predicting the values
train_prediction = scaler.inverse_transform(train_predictions).flatten() # Inverse transforming the predicted values

y = scaler.inverse_transform("y_testing_data_values".reshape(-1, 1)).flatten() # Inverse transforming the y values

train_results = pd.DataFrame(data={'Train Predictions': train_prediction, 'Actuals': y}) # Creating a dataframe
print(train_results)  # Displaying the results

# Saving the Trained Model

Now, we have to save the moedel so we can load the model without re-training the model.

In [ ]:
model.save("path_to_save_the_maodel") # Saving the model

In [ ]:
model1 = load_model("path_to_save_the_maodel") # Loading the model

# Loss Values of the Model 

Now , we have to check the loss values of the Model we trained so we can get the  accuracy of the model.

In [ ]:
rmse = math.sqrt(mean_squared_error(y, train_prediction))
mae = mean_absolute_error(y, train_prediction)
mape = mean_absolute_percentage_error(train_prediction, y)

# Displaying the RMSE, MAE and MAPE values

print('RMSE:', rmse)
print('MAE:', mae)
print('MAPE:', mape)

In [ ]:
# Loss  Graph for the model
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left')
plt.show()